In [1]:
import cohere
import asyncio
import nest_asyncio

In [10]:
co = cohere.AsyncClient("") 

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

result_df = pd.read_csv('PaLM2input.csv', index_col=0)

result_df['GPT_input'] = result_df['GPT_input'].str.replace('minimum', 'min')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maximum', 'max')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('average', 'avg')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maxntprobnp', 'max ntprobnp')

result_df['GPT-Diagnoses'] = np.nan
result_df['GPT-Eval'] = np.nan

In [4]:
prompt_no_example = '''Suggest as many potential diagnoses as possible from patient data.
In addition, include previously diagnosed conditions and information about patient's medical history (if any). 
Give exact numbers and/or text quotes from the data that made you think of each of the diagnoses \
and, if necessary, give further tests that could confirm the diagnosis. 
Once you're done, suggest further, more complex diseases that may be ongoing based on the existing diagnoses you already made. 
Use the International Classification of Disease (ICD) naming standard for reporting the diagnoses, but you don't have to specify the codes.
Use the same spacing as punctuation in your answer as in the example below.

Before finalizing your answer check if you haven't missed any abnormal data points and hence any diagnoses that could be made \
based on them. If you did, add them to your list of diagnoses.'''

example = '''

For example, if the patient data mentions:

"Blood report: 
min glucose: 103, max glucose: 278, avg glucose: 156.5, max inr: 2.1, max pt: 22.4, max ptt: 150, \
avg wbc: 13.8, max wbc: 14.1, max lactate: 5.9, max bun: 101, max creatinine: 5.8, avg bun: 38.15, avg creatinine: 2.78
Blood gas report: 
3 hours after admission the blood gas results from venous blood are: ph: 7.2
Imaging report: 
Status post left total shoulder replacement
Chest X-Ray Possible small right pleural effusion and Mild, bibasilar atelectasis.. \
Lung volumes have slightly increased but areas of atelectasis are seen at both the left and the right lung bases
Microbiology tests: 
24 hours after admission the microbiology culture test MRSA SCREEN obtained via MRSA SCREEN \
identified POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS
Vitalsigns data from ICU: 
max temperature: 38, min peripheral oxygen saturation: 70, max respiration rate: 29"

then your answer may be: 

1: Methicillin resistant Staphylococcus aureus infection, site unspecified
Foundational data: Microbiology culture test identifying "POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS"

2: Atelectasis 
Foundational data from Chest X-Ray: "Mild, bibasilar atelectasis.. \
Lung volumes have slightly increased but areas of atelectasis are seen at both the left and the right lung bases"

3: Pleural effusion, not elsewhere classified
Foundational data from Chest X-Ray: "Possible small right pleural effusion."
Further tests: Thoracentesis, CT chest

4: Acidosis
Foundational data: "ph: 7.2"
Further tests: Urine pH, Anion Gap

5: Lactic acidosis
Foundational data: "max lactate: 5.9"

6: Acquired coagulation factor deficiency
Foundational data: "max inr: 2.1, max pt: 22.4, max ptt: 150"
Further tests: Antiphospholipid Antibodies (APL), Protein C, Protein S, Antithrombin III, Factor V Leiden, Fibrinogen test

7: Hyperglycemia, unspecified
Foundational data: "max glucose: 278, avg glucose: 156.5".
Further tests: Hemoglobin A1c (HbA1c) test

8: Hypoxemia
Foundational data: "min peripheral oxygen saturation: 70"
Further tests: Measure PaO2 in blood

9: Leukocytosis
Foundational data: "max wbc: 14.1, avg wbc: 13.8". The patient's white blood cell count is consistently elevated which may suggest an ongoing inflammatory response or infection.
Further tests: Infection markers such as CRP or PCT, Assessment of symptoms like fever, inflammation or fatigue. 

10. Unspecified acute kidney failure:
Foundational data: "max bun: 101, max creatinine: 5.8, avg bun: 38.15, avg creatinine: 2.78"
Further tests: Urine output measurements for oliguria, ultrasound to rule out obstruction

11. Presence of left artificial shoulder joint
Foundational data: The imaging report mentions: "Status post left total shoulder replacement"

Further diseases based on these diagnoses (continued the indexing from the previous number in the list):

12: Unspecified septicemia
Foundational data: positive MRSA screen, systemic inflammatory response: "max respiration rate: 29", "max temperature: 38", leukocytosis
Further tests: HR, BP, wound culture, respiratory excretion tests

13: Septic shock
Foundational data: Septicemia with acidosis and lactic acidosis may suggest septic shock
Further tests: patient examination (low BP, mental disorientation, nausea, pale skin may confirm the finding)

14: Acute respiratory failure, with hypoxia or hypercapnia
Foundational data: hypoxemia and the presence of atelectasis
Further tests: Clinical symptoms (severe shortness of breath, rapid breathing, and confusion), \
arterial blood gas measurements showing hypoxia or hypercapnia

15: Type 2 diabetes mellitus with diabetic chronic kidney disease
Foundational data: Hyperglycemia and kidney failure
Further tests: urine test, hemoglobin (A1C) test, GFR, BP, physical examination (swelling, nausea, weakness, eye disease)'''

In [5]:
nest_asyncio.apply()

In [11]:
pre = prompt_no_example + example

async def get_response(msg):
    response = await co.chat(message=msg,
                       model="command-r-plus",
                       preamble=pre)
    return response

async def get_diagnoses(queries):
    loop = asyncio.get_event_loop()
    tasks = []
    for query in queries:
        tasks.append(get_response(query))
    all_data = loop.run_until_complete(asyncio.gather(*tasks))
    return all_data

In [8]:
query_per_call = 10 #roughly around 40000 tokens / min (limit) at max
repeat = []
for i in tqdm(range(0,1000,query_per_call)):
    try:
        idx_from = i
        idx_to = i + query_per_call
        queries = ['Patient data:\n' +  result_df.iloc[i]['GPT_input'] for i in range(idx_from,idx_to)]
        result = asyncio.run(get_diagnoses(queries))
        contents = [result[i].text for i in range(query_per_call)]
        hadm_ids = [result_df.index[i] for i in range(idx_from,idx_to)]
        result_df.loc[hadm_ids, 'GPT-Diagnoses'] = contents
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        repeat.append(i)
        print(e)

 47%|█████████████████████████████████████▏                                         | 47/100 [40:35<1:00:41, 68.71s/it]

Error happened at iteration i: 460
status_code: 502, body: 


 89%|██████████████████████████████████████████████████████████████████████▎        | 89/100 [1:10:54<03:55, 21.39s/it]

Error happened at iteration i: 870
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error happened at iteration i: 880
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 1000 API calls / month. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


 91%|███████████████████████████████████████████████████████████████████████▉       | 91/100 [1:10:54<01:35, 10.56s/it]

...

Error happened at iteration i: 950
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 40 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error happened at iteration i: 960
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 40 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


 99%|██████████████████████████████████████████████████████████████████████████████▏| 99/100 [1:10:55<00:00,  1.37it/s]

Error happened at iteration i: 970
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 40 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"
Error happened at iteration i: 980
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 40 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:10:56<00:00, 42.56s/it]

Error happened at iteration i: 990
status_code: 429, body: data=None message="You are using a Trial key, which is limited to 40 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"


In [12]:
query_per_call = 10 #roughly around 40000 tokens / min (limit) at max
for i in tqdm(repeat):
    try:
        idx_from = i
        idx_to = i + query_per_call
        queries = ['Patient data:\n' +  result_df.iloc[i]['GPT_input'] for i in range(idx_from,idx_to)]
        result = asyncio.run(get_diagnoses(queries))
        contents = [result[i].text for i in range(query_per_call)]
        hadm_ids = [result_df.index[i] for i in range(idx_from,idx_to)]
        result_df.loc[hadm_ids, 'GPT-Diagnoses'] = contents
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        print(e)

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [09:52<00:00, 42.29s/it]


In [15]:
result_df.to_csv('Cohere_result.csv')